In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report

In [2]:
VOCAB_SIZE = 5000
MAX_LEN = 400
BATCH_SIZE = 32
EMBED_DIM = 50
FILTERS = 250
N_CLASS = 2
N_EPOCH = 2
DISPLAY_STEP = 50

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=VOCAB_SIZE)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=MAX_LEN)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=MAX_LEN)

In [4]:
def pipeline_train(X, y, sess):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(len(X)).batch(BATCH_SIZE)
    iterator = dataset.make_initializable_iterator()
    X_ph = tf.placeholder(tf.int32, [None, MAX_LEN])
    y_ph = tf.placeholder(tf.int64, [None])
    init_dict = {X_ph: X, y_ph: y}
    sess.run(iterator.initializer, init_dict)
    return iterator, init_dict

def pipeline_test(X, sess):
    dataset = tf.data.Dataset.from_tensor_slices(X)
    dataset = dataset.batch(BATCH_SIZE)
    iterator = dataset.make_initializable_iterator()
    X_ph = tf.placeholder(tf.int32, [None, MAX_LEN])
    init_dict = {X_ph: X}
    sess.run(iterator.initializer, init_dict)
    return iterator, init_dict


sess = tf.Session()
iter_train, init_dict_train = pipeline_train(X_train, y_train, sess)
iter_test, init_dict_test = pipeline_test(X_test, sess)

In [5]:
def forward(x, reuse, is_training):
    with tf.variable_scope('model', reuse=reuse):
        x = tf.contrib.layers.embed_sequence(x, VOCAB_SIZE, EMBED_DIM)
        x = tf.layers.dropout(x, 0.2, training=is_training)
        feat_map = []
        for k_size in [3, 4, 5]:
            _x = tf.layers.conv1d(x, FILTERS, k_size, activation=tf.nn.relu)
            _x = tf.layers.max_pooling1d(_x, _x.get_shape().as_list()[1], 1)
            _x = tf.reshape(_x, (tf.shape(x)[0], FILTERS))
            feat_map.append(_x)
        x = tf.concat(feat_map, -1)
        x = tf.layers.dense(x, FILTERS, tf.nn.relu)
        logits = tf.layers.dense(x, N_CLASS)
    return logits

In [6]:
ops = {}

X_train_batch, y_train_batch = iter_train.get_next()
X_test_batch = iter_test.get_next()

logits_train_batch = forward(X_train_batch, reuse=False, is_training=True)
ops['pred_logits'] = forward(X_test_batch, reuse=True, is_training=False)

ops['global_step'] = tf.Variable(0, trainable=False)
ops['lr'] = tf.train.exponential_decay(5e-3, ops['global_step'], 1400, 0.2)

ops['loss'] = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=logits_train_batch, labels=y_train_batch))

ops['train'] = tf.train.AdamOptimizer(ops['lr']).minimize(
    ops['loss'], global_step=ops['global_step'])

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [7]:
sess.run(tf.global_variables_initializer())
for epoch in range(1, N_EPOCH+1):
    while True:
        try:
            sess.run(ops['train'])
        except tf.errors.OutOfRangeError:
            break
        else:
            step = sess.run(ops['global_step'])
            if step % DISPLAY_STEP == 0 or step == 1:
                loss, lr = sess.run([ops['loss'], ops['lr']])
                print("Epoch %d | Step %d | Loss %.3f | LR: %.4f" % (epoch, step, loss, lr))
    
    y_pred_li = []
    while True:
        try:
            y_pred_li.append(sess.run(ops['pred_logits']))
        except tf.errors.OutOfRangeError:
            break
    y_pred = np.argmax(np.vstack(y_pred_li), 1)
    print("\nAccuracy: %.4f\n" % (y_pred==y_test).mean())
    
    if epoch != N_EPOCH:
        sess.run(iter_train.initializer, init_dict_train)
        sess.run(iter_test.initializer, init_dict_test)

Epoch 1 | Step 1 | Loss 0.724 | LR: 0.0050
Epoch 1 | Step 50 | Loss 0.815 | LR: 0.0047
Epoch 1 | Step 100 | Loss 0.576 | LR: 0.0045
Epoch 1 | Step 150 | Loss 0.395 | LR: 0.0042
Epoch 1 | Step 200 | Loss 0.225 | LR: 0.0040
Epoch 1 | Step 250 | Loss 0.307 | LR: 0.0038
Epoch 1 | Step 300 | Loss 0.223 | LR: 0.0035
Epoch 1 | Step 350 | Loss 0.144 | LR: 0.0033
Epoch 1 | Step 400 | Loss 0.181 | LR: 0.0032
Epoch 1 | Step 450 | Loss 0.248 | LR: 0.0030
Epoch 1 | Step 500 | Loss 0.279 | LR: 0.0028
Epoch 1 | Step 550 | Loss 0.423 | LR: 0.0027
Epoch 1 | Step 600 | Loss 0.282 | LR: 0.0025
Epoch 1 | Step 650 | Loss 0.235 | LR: 0.0024
Epoch 1 | Step 700 | Loss 0.315 | LR: 0.0022
Epoch 1 | Step 750 | Loss 0.172 | LR: 0.0021

Accuracy: 0.8850

Epoch 2 | Step 800 | Loss 0.127 | LR: 0.0020
Epoch 2 | Step 850 | Loss 0.419 | LR: 0.0019
Epoch 2 | Step 900 | Loss 0.186 | LR: 0.0018
Epoch 2 | Step 950 | Loss 0.162 | LR: 0.0017
Epoch 2 | Step 1000 | Loss 0.093 | LR: 0.0016
Epoch 2 | Step 1050 | Loss 0.080 | LR: